## Export Coco dataset

In [ ]:
import datetime
import json
import os
import random
import shutil
from PIL import Image as PImage

In [ ]:
PATH_DATASET = "./data"
PATH_COCORDIAIS = "./cocordiais"

IMAGES_METFACES = os.path.join(PATH_DATASET, "metfaces-source")
IMAGES_CORDIAIS = os.path.join(PATH_DATASET, "cordiais-source")

JSON_METFACES = os.path.join(PATH_DATASET, "metfaces.json")
JSON_CORDIAIS = os.path.join(PATH_DATASET, "cordiais.json")

FILES_METFACES = sorted([f.replace(".jpg", "") for f in os.listdir(IMAGES_METFACES) if f.endswith(".jpg")])
FILES_CORDIAIS = sorted([f.replace(".jpg", "") for f in os.listdir(IMAGES_CORDIAIS) if f.endswith(".jpg")])

random.seed(101010)
shuffled_cordiais = random.sample(FILES_CORDIAIS, k=len(FILES_CORDIAIS))
shuffled_metfaces = random.sample(FILES_METFACES, k=len(FILES_METFACES))

INITIAL_DATASETS = [
  {
    "name": "cordiais",
    "license_id": 1,
    "date": "2023-06-10 00:00:00",
    "splits": {
      "test": shuffled_cordiais[int(0.5 * len(FILES_CORDIAIS)):],
      "train": shuffled_cordiais[int(0.1 * len(FILES_CORDIAIS)) : int(0.5 * len(FILES_CORDIAIS))],
      "validation": shuffled_cordiais[:int(0.1 * len(FILES_CORDIAIS))]
    }
  },
  {
    "name": "metfaces",
    "license_id": 1,
    "date": "2020-06-10 00:00:00",
    "splits": {
      "test": [],
      "train": shuffled_metfaces[int(0.2 * len(FILES_METFACES)):],
      "validation": shuffled_metfaces[:int(0.2 * len(FILES_METFACES))]
    }
  }
]


### Copy files to cocordiais directory

In [ ]:
for dset in INITIAL_DATASETS:
  for split in dset["splits"].keys():
    for fn in dset["splits"][split]:
      shutil.copy2(
        os.path.join(PATH_DATASET, "%s-source" % dset["name"], "%s.jpg" % fn),
        os.path.join(PATH_COCORDIAIS, split)
      )

shutil.copy2(
  os.path.join(PATH_DATASET, "id2label.json"),
  os.path.join(PATH_COCORDIAIS)
)

shutil.copy2(
  os.path.join(PATH_DATASET, "label2id.json"),
  os.path.join(PATH_COCORDIAIS)
)


In [ ]:
cocordiais_dataset = {
  "info": {
    "year": 2023,
    "version": "1.0",
    "description": "Object Detection dataset to detect female-ish faces in paintings",
    "contributor": "Thiago Hersan",
    "url": "https://huggingface.co/thiagohersan",
    "date_created": "%s" % datetime.datetime.now(),
  },
  "categories": [
    { "id": 0, "name": "N/A", "supercategory": "N/A", },
    { "id": 1, "name": "female", "supercategory": "face", },
    { "id": 2, "name": "not-female", "supercategory": "face", },
  ],
  "licenses": [
    { "id": 1, "name": "CC BY-NC 2.0", "url": "https://creativecommons.org/licenses/by-nc/2.0/", },
    { "id": 2, "name": "CC0 1.0", "url": "https://creativecommons.org/publicdomain/zero/1.0/", },
  ],
  "images": [],
  "annotations": [],
}

In [ ]:
# TODO: this could be a class... maybe ?
files = {}
img2id = {}
id2img = {}
coco_annotations = {}

for s in ["test", "train", "validation"]:
  split_dir = os.path.join(PATH_COCORDIAIS, s)
  split_file_list = sorted([f.replace(".jpg", "") for f in os.listdir(split_dir) if f.endswith(".jpg")])
  files[s] = set(split_file_list)
  img2id[s] = {f: i for i,f in enumerate(split_file_list)}
  id2img[s] = {}
  coco_annotations[s] = []


def get_split(fn):
  for s in ["test", "train", "validation"]:
    if fn in files[s]:
      return s


for dset in INITIAL_DATASETS:
  dset_name = dset["name"]
  dataset_path_in = os.path.join(PATH_DATASET, "%s.json" % dset_name)

  with open(dataset_path_in) as json_file_in:
    data = json.load(json_file_in)
    for obj in data:
      my_split = get_split(obj["source_image"])
      my_img_id = img2id[my_split][obj["source_image"]]

      id2img[my_split][my_img_id] = {
        "id": my_img_id,
        "width": obj["source_image_w"],
        "height": obj["source_image_h"],
        "file_name": "%s.jpg" % obj["source_image"],
        "license": dset["license_id"],
        "date_captured": dset["date"],
        "coco_url": "",
        "flickr_url": ""
      }

      poly = [
        [obj["face_rect_xyxy"][0], obj["face_rect_xyxy"][1]],
        [obj["face_rect_xyxy"][2], obj["face_rect_xyxy"][1]],
        [obj["face_rect_xyxy"][2], obj["face_rect_xyxy"][3]],
        [obj["face_rect_xyxy"][0], obj["face_rect_xyxy"][3]]
      ]

      ann_obj = {
        "id": len(coco_annotations[my_split]),
        "image_id": my_img_id,
        "category_id": obj["gender"],
        "segmentation": list([poly]),
        "area": obj["face_rect_xywh"][2] * obj["face_rect_xywh"][3],
        "bbox": obj["face_rect_xywh"],
        "iscrowd": 0,
      }

      coco_annotations[my_split].append(ann_obj)

for s in ["test", "train", "validation"]:
  coco_split = json.loads(json.dumps(cocordiais_dataset))
  coco_split["images"] = list(id2img[s].values())
  coco_split["annotations"] = list(coco_annotations[s])

  out_file = os.path.join(PATH_COCORDIAIS, s, "cocordiais.json")

  with open(out_file, 'w') as json_file_out_write:
    json.dump(coco_split, json_file_out_write)


### Check jsons, images and annotations

In [ ]:
import torchvision
import os
import random

from PIL import Image, ImageDraw
from transformers import DetrImageProcessor

In [ ]:
class CocoDetection(torchvision.datasets.CocoDetection):
  def __init__(self, img_folder, processor, train=True):
    ann_file = os.path.join(img_folder, "cocordiais.json")
    super(CocoDetection, self).__init__(img_folder, ann_file)
    self.processor = processor

  def __getitem__(self, idx):
    # read in PIL image and target in COCO format
    # feel free to add data augmentation here before passing them to the next step
    img, target = super(CocoDetection, self).__getitem__(idx)
    
    # preprocess image and target (converting target to DETR format, resizing + normalization of both image and target)
    image_id = self.ids[idx]
    target = {'image_id': image_id, 'annotations': target}
    encoding = self.processor(images=img, annotations=target, return_tensors="pt")
    pixel_values = encoding["pixel_values"].squeeze() # remove batch dimension
    target = encoding["labels"][0] # remove batch dimension

    return pixel_values, target

In [ ]:
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")

train_dataset = CocoDetection(img_folder='./cocordiais/train', processor=processor)
val_dataset = CocoDetection(img_folder='./cocordiais/validation', processor=processor, train=False)

print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(val_dataset))

### Show images

In [ ]:
image_ids = train_dataset.coco.getImgIds()
image_id = 2
image_id = 3
image_id = random.randint(0, len(image_ids))

image = train_dataset.coco.loadImgs(image_id)[0]
image = Image.open(os.path.join('./cocordiais/train', image['file_name']))

annotations = train_dataset.coco.imgToAnns[image_id]
draw = ImageDraw.Draw(image, "RGBA")

cats = train_dataset.coco.cats
id2label = {k: v['name'] for k,v in cats.items()}

for annotation in annotations:
  label = id2label[annotation['category_id']]
  x,y,w,h = tuple(annotation['bbox'])
  draw.rectangle((x,y,x+w,y+h), outline='red', width=2)
  draw.text((x, y), label, fill='white')

image